In [1]:
import exifread
import matplotlib.colors
import matplotlib.figure
import matplotlib.pyplot
import matplotlib.backends.backend_wxagg
import numpy
import PIL.Image
import sys
import wx
import wx.lib.floatcanvas.FloatCanvas

In [2]:
def image_import():
    image_path = "../SEM Images/Armin245.tif"
    image = PIL.Image.open(image_path)
    
    exif_file = open(image_path, 'rb')
    tags = exifread.process_file(exif_file, strict = False)
    if "Image Tag 0x8546" in tags.keys():
        print("Successful, found SEM information in image file.")

        # SEM information metadata is a very long binary string and includes data 
        #  that seem to be non decodable ASCII / UTF-8. The information needed is 
        #  in the first 100 characters, so everything after that is ignored.
        stuff = tags["Image Tag 0x8546"]
        stuff_value = stuff.values
        stuff_value = stuff_value[0:100].decode('utf-8')
        stuff_value_list = stuff_value.splitlines()
        
        pixelsize = stuff_value_list[3]
        magnification = stuff_value_list[4]
    else:
        print("Failed, could not find SEM information in image file.")
        sys.exit(0)
    
    return image

def image_fft_numpy(image):
    image_array = numpy.asarray(image)
    image_array_fft = numpy.fft.fft2(image_array)
    image_array_fft = numpy.fft.fftshift(image_array_fft)
    return image_array_fft

In [3]:
image = image_import()
image_array = numpy.asarray(image)
image_array_fft = image_fft_numpy(image) 

figure = matplotlib.figure.Figure()
plot1, plot2 = figure.subplots(1, 2)
im1 = plot1.imshow(image_array, cmap = 'gray')
plot1.set_title("SEM Image")
im2 = plot2.imshow(numpy.abs(image_array_fft), norm = matplotlib.colors.LogNorm())
plot2.set_title("SEM Image FFT")
figure.colorbar(im2)

app = wx.App()
frame = wx.Frame(None, title = "SEM Realtime Diagnostic Tool")
canvas = matplotlib.backends.backend_wxagg.FigureCanvasWxAgg(frame, -1, figure)
canvas.draw()
frame.Show()
app.MainLoop()

Possibly corrupted field Tag 0x8546 in Image IFD
Successful, found SEM information in image file.


0